In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta

df = pd.read_csv('train.csv')
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")

df.set_index('Date', inplace=True)

df['day_of_week'] = df.index.dayofweek
df['day_of_month'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

X = df[['day_of_week', 'day_of_month', 'month', 'year']]
y = df['MyWay']

model = LinearRegression()

model.fit(X, y)

last_date = df.index[-1]
next_dates = [last_date + timedelta(days=i) for i in range(1, 8)]

next_days = pd.DataFrame({'Date': next_dates})
next_days['day_of_week'] = next_days['Date'].dt.dayofweek
next_days['day_of_month'] = next_days['Date'].dt.day
next_days['month'] = next_days['Date'].dt.month
next_days['year'] = next_days['Date'].dt.year

forecast = model.predict(next_days[['day_of_week', 'day_of_month', 'month', 'year']])

forecast_df = pd.DataFrame({'Date': next_dates, 'Forecast': forecast})

# plt.plot(df.index, df['MyWay'], label='Actual')
# plt.plot(forecast_df['Date'], forecast_df['Forecast'], label='Forecast', color='red')
# plt.xlabel('Date')
# plt.ylabel('MyWay')
# plt.title('Linear Regression Forecast for Next 7 Days')
# plt.legend()
# plt.show()

print("\nPredicted Output:")
print(forecast_df)

print("\nActual Output:")
print(df.tail(7))
from sklearn.metrics import r2_score

y_pred = model.predict(X)

r2 = r2_score(y, y_pred)

print("R-squared Score:", r2)



Predicted Output:
        Date      Forecast
0 2023-07-27  42175.619313
1 2023-07-28  35949.511505
2 2023-07-29  29723.403696
3 2023-07-30  23497.295887
4 2023-07-31  60409.357736
5 2023-08-01  56173.574424
6 2023-08-02  49947.466615

Actual Output:
            MyWay  Paper Ticket  day_of_week  day_of_month  month  year
Date                                                                   
2022-11-14  55417          6715            0            14     11  2022
2020-09-28  33943          2085            0            28      9  2020
2023-01-19  46083          4622            3            19      1  2023
2023-08-16  70115          6955            2            16      8  2023
2022-01-10  21932          2315            0            10      1  2022
2020-05-04  14842           854            0             4      5  2020
2023-07-26  69067          6811            2            26      7  2023
R-squared Score: 0.28650583931670837


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv('train.csv')
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")

df.set_index('Date', inplace=True)

model = ARIMA(df['MyWay'], order=(5,1,0))

model_fit = model.fit()

forecast = model_fit.forecast(steps=7)

last_date = df.index[-1]
next_dates = [last_date + timedelta(days=i) for i in range(1, 8)]

forecast_df = pd.DataFrame({'Date': next_dates, 'Forecast': forecast})

# plt.plot(df.index, df['MyWay'], label='Actual')
# plt.plot(forecast_df['Date'], forecast_df['Forecast'], label='Forecast', color='red')
# plt.xlabel('Date')
# plt.ylabel('MyWay')
# plt.title('ARIMA Forecast for Next 7 Days')
# plt.legend()
# plt.show()

actual_values = df.iloc[-7:]
actual = actual_values['MyWay'].values
forecast = forecast_df['Forecast'].values

mae = mean_absolute_error(actual, forecast)

mse = mean_squared_error(actual, forecast)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

print("\nActual Output:")
print(actual_values)

print("\nPredicted Output:")
print(forecast_df)

mape = np.mean(np.abs((actual - forecast) / actual)) * 100

bias = np.mean(actual - forecast)

print("Mean Absolute Percentage Error (MAPE):", mape)
print("Forecast Bias:", bias)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsm

Mean Absolute Error (MAE): 18728.85518913806
Mean Squared Error (MSE): 472696952.8290419
Root Mean Squared Error (RMSE): 21741.59499275621

Actual Output:
            MyWay  Paper Ticket
Date                           
2022-11-14  55417          6715
2020-09-28  33943          2085
2023-01-19  46083          4622
2023-08-16  70115          6955
2022-01-10  21932          2315
2020-05-04  14842           854
2023-07-26  69067          6811

Predicted Output:
           Date      Forecast
1778 2023-07-27  42263.425883
1779 2023-07-28  42518.402758
1780 2023-07-29  45463.479126
1781 2023-07-30  39154.250173
1782 2023-07-31  41178.587779
1783 2023-08-01  47206.656920
1784 2023-08-02  42885.505950
Mean Absolute Percentage Error (MAPE): 62.60366105349628
Forecast Bias: 1532.6702013542806


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
